In [1]:
import os
import sys
from itertools import groupby, product
import urllib.request as request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
torch.cuda.set_device(0)

np.set_printoptions(suppress=True)

pd.set_option('display.max_columns',   10)
pd.set_option('display.max_rows',    1000)
pd.set_option('display.max_colwidth', 150)

SEED    = 420
np.random.seed(SEED)

AA      = 'AGILPVFWYDERHKSTCMNQ'

AA_FREQ = np.array(
          [0.08259085, 0.07077786, 0.05966563, 0.09670638, 0.04705176,
           0.06877565, 0.03864251, 0.01081189, 0.02923216, 0.05456002,
           0.06757433, 0.05536090, 0.02272500, 0.05846431, 0.06567224,
           0.05345880, 0.01371509, 0.02422665, 0.04064471, 0.03934328], dtype=float)

In [2]:
def kfold(y, nsplits=10):
    # deterministic algorithm, no random seed used
    s = groupby(sorted(i[::-1] for i in enumerate(y)),key=lambda x: x[0])
    s = [[j[1] for j in i[1]] for i in s]
    q = []
    for i in s:
        t = len(i)
        r = t%nsplits
        a = [int((t-r)/nsplits)]*nsplits
        for j in range(r):
            a[j] = a[j]+1
        q += [[[i.pop() for k in range(j)] for j in a]]
        assert len(i)==0
    a = [i[0]+i[1] for i in zip(*q)]
    for i in range(nsplits):
        o = filter(lambda x:x!=i,range(nsplits))
        yield np.array(a[i]), np.array([k for j in o for k in a[j]])

def grid_search_summary(search):
    f = [i for i in search.cv_results_.keys() if i.startswith('split0')]
    f = [[i.replace('split0','mean') , i.replace('split0','std') ] for i in f]
    f = [[j[0][10:], np.array([search.cv_results_[i] for i in j]).T] for j in f]
    f = {i: ['%.1f ± %.1f' % tuple(k*100) for k in j] for i, j in f}
    df = pd.DataFrame(f)
    df['params'] = search.cv_results_['params']
    df['rank_test_accuracy'] = search.cv_results_['rank_test_accuracy']
    return df.sort_values(by=['rank_test_accuracy'])

class AAindex(dict):
    def __init__(self, dir='/tmp', update=False):
        dir = self._retrieve(dir+'/', update=update)
        for i in self._build(open(dir+'aaindex/aaindex1')):
            self[i['H']] = i
        for i in self._build(open(dir+'aaindex/aaindex2')):
            self[i['H']] = i
        # for i in self._build(open(dir+'aaindex/aaindex3')):
        #     self[i['H']] = i
        return 
    
    def _retrieve(self, dir, update=False):
        url = 'ftp://ftp.genome.jp/pub/db/community/aaindex/'
        aai = ('aaindex1','aaindex2','aaindex3','aaindex.doc')
        if not os.path.isdir(dir+'aaindex'):
            os.mkdir(dir+'aaindex')
        elif update:
            pass
        else:
            c = os.listdir(dir+'aaindex')
            if all(_ in c for _ in aai):
                return dir
        for f in aai:
            response = request.urlopen(url+f)
            with open(dir+'aaindex/'+f, 'wb') as w:
                w.write(response.read())
        return dir
    
    def _build(self, stream):
        def get_blocks(x):
            it = groupby(x, lambda _: _[0])
            bef, out = next(it)
            out = ''.join(out)
            for k, i in it:
                if k==' ':
                    out += ''.join(i)
                elif bef!=k:
                    yield out
                    bef = k
                    out = ''.join(i)
                else:
                    raise Exception('never should have come here')
            yield out

        def reformat(x):
            floatna = lambda x: float('nan') if x=='NA' else float(x)
            floatd  = lambda x: float('nan') if x=='-' else float(x)
            for k in x:
                if k in 'HDRATJ':
                    x[k] = x[k].replace('\n','')
                elif k == 'C':
                    x[k] = x[k].split()
                    x[k] = dict((i,float(j)) for i,j in zip(x[k][::2],x[k][1::2]))
                elif k == 'I':
                    x[k] = x[k][x[k].index('\n'):].split()
                    x[k] = map(floatna, x[k])
                    x[k] = dict(zip('ARNDCQEGHILKMFPSTWYV',x[k]))
                    assert len(x[k])==20
                elif k == 'M':
                    rc,m = x[k].split('\n', 1)
                    r ,c = [_.split(" = ")[1] for _ in rc.split(',')]
                    m    = [list(map(floatd, j.split())) for j in [i for i in m.split('\n')]]
                    m    = [[(v, c[ci]+r[ri]) for ci, v in enumerate(_)] for ri, _ in enumerate(m)]
                    x[k] = dict(j[::-1] for i in m for j in i)
            return x
        
        get_entry  = lambda x: (i for k, i in groupby(x, lambda _: _.startswith('//')) if not k)
        _estrip    = lambda x: (x[0], '\n'.join(_[2:] for _ in x.strip().split('\n')))
        read_entry = lambda x: dict(map(_estrip, get_blocks(x)))

        for i in get_entry(stream):
            yield reformat(read_entry(i))
    
    def create_lookup_table(self, exclude=[]):
        aa = 'ARNDCQEGHILKMFPSTWYV'
        d, k = {i:[] for i in aa}, []
        for i in self:
            if 'I' not in self[i]:
                continue
            if i in exclude:
                continue
            k += [i]
            for j in aa:
                d[j] += [self[i]['I'][j]]
        return d, k

    def featurize(self, seqs, prefix='AAi', exclude=[]):
        assert 1==len(set(map(len,seqs)))
        dic, key = self.create_lookup_table(exclude=exclude)
        lab = ['%s:%s:%s'%(prefix,n+1,j) for n, i in enumerate(seqs[0]) for j in key]
        x   = np.array(list(map(lambda x: np.concatenate([dic[i] for i in x]), seqs)))
        return x, lab

AAi = AAindex(update=False)


# load full dataset

In [3]:
from sklearn.decomposition import PCA

def load_features():
    # load all datasets
    esm_pt  = 'data/transformer/representations_caax_pad100.pt'
    esm_fa  = 'data/transformer/caax_pad100.fa'
    esm_tsr = torch.load(esm_pt)[:,-5:,:]
    
    esm_seq = np.array([i.strip()[-4:] for i in open(esm_fa) if not i.startswith('>')])
    
    df      = pd.read_csv('data/dataset_scaled.csv')
    df      = df.set_index('sequence').loc[esm_seq]
    
    assert all(df.index.values==esm_seq)
    
    # create individual datasets
    seq     = esm_seq
    prenyl  = df['prenyl'].values
    cleave  = df['cleave'].values    
    oh_lab  = np.array(list(filter(lambda x:x.startswith('OH:'), df.columns)), dtype=object)
    oh      = df[oh_lab].values#reset_index(drop=True)
    aai_lab = np.array(list(filter(lambda x:x.startswith('AAi:'), df.columns)), dtype=object)
    aai     = df[aai_lab].values#reset_index(drop=True)
    esm     = esm_tsr.reshape(esm_tsr.shape[0], esm_tsr.shape[1]*esm_tsr.shape[2])
    
    # shuffle
    shuffle = np.random.permutation(np.arange(8000))
    seq     = seq[shuffle]
    prenyl  = prenyl[shuffle]
    cleave  = cleave[shuffle]
    oh      = oh[shuffle]
    aai     = aai[shuffle]
    esm     = esm[shuffle]
    
    return seq, prenyl, cleave, oh, aai, esm

def load_validation(seq):
    val = pd.DataFrame(
        [['CIIS', 1, 1],
         ['CIKS', 0, 0],
         ['CIIL', 1, 1],
         ['CIDL', 0, 0],
         ['CVIM', 1, 1],
         ['CVKM', 1, 0],
         ['CSII', 1, 1],
         ['CSEI', 0, 0],
         ['CSNA', 1, 0],
         ['CYNA', 1, 0],
         ['CSGL', 0, 0],
         ['CSGK', 0, 0],
         ['CKQS', 1, 0],
         ['CFIF', 1, 0],
         ['CAPY', 1, 0],
         ['CTVA', 1, 1],
         ['CALD', 1, 0],
         ['CAVS', 1, 1],
         ['CIQF', 1, 0]],
        columns=['sequence','prenyl','cleave'])
    d = dict(i[::-1] for i in enumerate(seq))
    val_ind = [d[i] for i in val['sequence'].values]
    return val, val_ind

def decompose(oh, aai, esm, known):
    print('known: ',sum(known))
    var    = 0.99
    
    pca    = PCA(n_components=None, random_state=SEED).fit(oh[known])
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    oh_pc  = pca.transform(oh)[:,:(cumsum<var).sum()]
    
    pca    = PCA(n_components=None, random_state=SEED).fit(aai[known])
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    aai_pc = pca.transform(aai)[:,:(cumsum<var).sum()]
    
    pca    = PCA(n_components=None, random_state=SEED).fit(esm[known])
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    esm_pc = pca.transform(esm)[:,:(cumsum<var).sum()]   
    
    return oh_pc, aai_pc, esm_pc

seq, prenyl, cleave, oh, aai, esm = load_features()
val, val_ind                      = load_validation(seq)
oh_pc, aai_pc, esm_pc             = decompose(oh, aai, esm, prenyl==prenyl)

all_models                        = {'motif':seq}

known:  997


In [4]:
print(oh.shape[ 1], '\t', oh_pc.shape[ 1])
print(aai.shape[1], '\t', aai_pc.shape[1])
print(esm.shape[1], '\t', esm_pc.shape[1])

60 	 53
1659 	 50
6400 	 276


# train (PSSM)

In [5]:
def kfold(y, nsplits=10):
    # deterministic algorithm, no random seed used
    s = groupby(sorted(i[::-1] for i in enumerate(y)),key=lambda x: x[0])
    s = [[j[1] for j in i[1]] for i in s]
    q = []
    for i in s:
        t = len(i)
        r = t%nsplits
        a = [int((t-r)/nsplits)]*nsplits
        for j in range(r):
            a[j] = a[j]+1
        q += [[[i.pop() for k in range(j)] for j in a]]
        assert len(i)==0
    a = [i[0]+i[1] for i in zip(*q)]
    for i in range(nsplits):
        o = filter(lambda x:x!=i,range(nsplits))
        yield np.array(a[i]), np.array([k for j in o for k in a[j]])

class Motif:
    def __init__(self, seqs, alphabet=AA, background=AA_FREQ):
        self.seqs       = seqs
        self.alphabet   = alphabet
        self.background = background
        self.pssm       = self.get_pssm()
        self.cutoff     = float('nan')
        
    def __repr__(self):
        return '<PSSM seqs=%s cutoff=%.3f>' % (len(self.seqs),self.cutoff)
    
    def get_pfm(self):
        fxn = lambda x: [x.count(i) for i in self.alphabet]
        return np.array(list(zip(*map(fxn,zip(*self.seqs)))))

    def get_pwm(self):
        x = self.get_pfm()
        return x / x.sum(0)
    
    def get_pssm(self, pseudocount=0.05):
        x  = self.get_pfm()+pseudocount
        x /= x.sum(0)
        x  = x.T/self.background
        return np.log(x).T
    
    def score(self, seq):
        assert len(seq)==self.pssm.shape[1]
        x = map(self.alphabet.index,seq)
        return sum(s[i] for i,s in zip(x,self.pssm.T))
    
    def scores(self, seqs):
        return list(map(self.score, seqs))


In [6]:
def analysis():
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    
    q     = prenyl
    isnan = q == q

    X = seq[isnan]
    y = q[isnan]
    folds = kfold(y, nsplits=10)
    
    summary = {i:[] for i in ('accuracy','precision','recall','f1')}
    for test, train in folds:
        train_x = X[train]
        train_y = y[train]
        test_x  = X[test]
        test_y  = y[test]
        
        model = Motif(train_x[train_y==1]) # model for scoring
        
        s = np.array(model.scores(train_x))
        r = (min(s),max(s))
        
        hy1, hx = np.histogram(s[train_y==1], bins=300, range=r)
        hy2, hx = np.histogram(s[train_y==0], bins=300, range=r)
        hxc     = np.convolve([0.5,0.5],hx)[1:-1]
        acc     = np.array([sum(hy2[hxc<i])+sum(hy1[hxc>i]) for i in hx]) / (sum(hy2)+sum(hy1))
        model.cutoff  = hx[acc.argmax()] # cutoff for discretizing scores

        pred_y  = (model.scores(test_x) > model.cutoff).astype(int)
        summary['accuracy'] += [accuracy_score(test_y,pred_y)]
        summary['precision'] += [precision_score(test_y,pred_y)]
        summary['recall'] += [recall_score(test_y,pred_y)]
        summary['f1'] += [f1_score(test_y,pred_y)]
        
    for k in summary:
        a = np.array(summary[k])
        print('%.1f ± %.1f\t%s' % (a.mean()*100, a.std()*100, k))
    
    # TRAIN ON FULL SET
    train_x = X
    train_y = y

    model = Motif(train_x[train_y==1]) # model for scoring

    s = np.array(model.scores(train_x))
    r = (min(s),max(s))

    hy1, hx = np.histogram(s[train_y==1], bins=300, range=r)
    hy2, hx = np.histogram(s[train_y==0], bins=300, range=r)
    hxc     = np.convolve([0.5,0.5],hx)[1:-1]
    acc     = np.array([sum(hy2[hxc<i])+sum(hy1[hxc>i]) for i in hx]) / (sum(hy2)+sum(hy1))
    model.cutoff  = hx[acc.argmax()] # cutoff for discretizing scores

    
    test_x = val['sequence'].values
    test_y = val['prenyl'].values
    pred_y = (model.scores(test_x) > model.cutoff).astype(int)

    print()
    print(f'{round(accuracy_score(test_y,pred_y)*100,1)  }\t\tvalidation accuracy')
    print(f'{round(precision_score(test_y,pred_y)*100,1) }\t\tvalidation precision')
    print(f'{round(recall_score(test_y,pred_y)*100,1)    }\t\tvalidation recall')
    print(f'{round(f1_score(test_y,pred_y)*100,1)        }\t\tvalidation f1')

    df = val.copy()
    df['predicted'] = pred_y
    df['log(P)']    = [round(i,3) for i in model.scores(test_x)]
    print()
    print(sum(df['prenyl'].values == df['predicted'].values), df['prenyl'].shape[0])
    display(df[['sequence', 'prenyl', 'predicted', 'log(P)']])
    
    all_models['pssm'] = (model.scores(seq) > model.cutoff).astype(float)
    
analysis()

83.8 ± 3.3	accuracy
87.7 ± 3.5	precision
77.9 ± 5.9	recall
82.4 ± 3.8	f1

68.4		validation accuracy
100.0		validation precision
57.1		validation recall
72.7		validation f1

13 19


,sequence,prenyl,predicted,log(P)
0,CIIS,1,1,6.680
1,CIKS,0,0,-1.108
2,CIIL,1,1,5.128
3,CIDL,0,0,0.454
4,CVIM,1,1,8.048
5,CVKM,1,0,0.260
6,CSII,1,1,5.801
7,CSEI,0,0,1.979
8,CSNA,1,0,3.827
9,CYNA,1,0,3.340


# train (SVM)

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

def grid_search_summary(search):
    f = [i for i in search.cv_results_.keys() if i.startswith('split0')]
    f = [[i.replace('split0','mean') , i.replace('split0','std') ] for i in f]
    f = [[j[0][10:], np.array([search.cv_results_[i] for i in j]).T] for j in f]
    f = {i: ['%.1f ± %.1f' % tuple(k*100) for k in j] for i, j in f}
    df = pd.DataFrame(f)
    df['params'] = search.cv_results_['params']
    df['rank_test_accuracy'] = search.cv_results_['rank_test_accuracy']
    return df.sort_values(by=['rank_test_accuracy'])


In [8]:
from sklearn.svm import SVC

def parameter_search(X, y):
    known = y == y
    
    # parameter optimization
    model  = SVC(random_state=SEED)
    param  = {
        'kernel' : ['linear'], #['linear', 'rbf', 'sigmoid'], 
        'C':       [400],      #[1, 5, 10, 15, 20, 25, 30, 40, 50, 80, 100, 200, 250, 300, 350, 400, 450, 500, 800, 1000],
        'gamma':   ['auto']
        }
    
    search = GridSearchCV(estimator = model, param_grid = param, verbose=1,
        scoring=('accuracy','precision','recall','f1'), refit='accuracy', n_jobs=23, cv=10)
    search.fit(X[known], y[known])
    return search

def validate(X, y):
    known = y == y
    
    # train
    param = {'C': 400, 'gamma': 'auto', 'kernel': 'linear'}
    model = SVC(random_state=SEED, **param)
    model.fit(X[known], y[known])
    
    # test
    y_pred = model.predict(X[val_ind])
    val['predict'] = y_pred.astype(int)

    # print('accuracy  :', accuracy_score(y_pred[validation], vali['observed'].values))
    # print('precision :', precision_score(y_pred[validation], vali['observed'].values))
    # print('recall    :', recall_score(y_pred[validation], vali['observed'].values))
    # print('f1        :', f1_score(y_pred[validation], vali['observed'].values))
    # print(y_pred[validation], vali['observed'].values)
    
    print('correct:', sum(val['prenyl'].astype(int) == val['predict']))
    
    all_models['svm_sequence'] = model.predict(X)
    return val

X      = oh_pc
y      = prenyl
search = parameter_search(X, y)
display(grid_search_summary(search).head(5))
validate(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=23)]: Using backend LokyBackend with 23 concurrent workers.
[Parallel(n_jobs=23)]: Done  10 out of  10 | elapsed:    2.9s finished


,accuracy,precision,recall,f1,params,rank_test_accuracy
0,86.0 ± 2.7,86.5 ± 4.0,84.9 ± 3.8,85.6 ± 2.8,"{'C': 400, 'gamma': 'auto', 'kernel': 'linear'}",1


correct: 16


,sequence,prenyl,cleave,predict
0,CIIS,1,1,1
1,CIKS,0,0,0
2,CIIL,1,1,1
3,CIDL,0,0,0
4,CVIM,1,1,1
5,CVKM,1,0,0
6,CSII,1,1,1
7,CSEI,0,0,0
8,CSNA,1,0,1
9,CYNA,1,0,0


In [9]:
from sklearn.svm import SVC

def parameter_search(X, y):
    known = y == y
    
    # parameter optimization
    model  = SVC(random_state=SEED)
    param  = {
        'kernel' : ['rbf'], #['rbf', 'sigmoid'], 
        'C':       [2], #[0.001, 0.005, 0.1, 0.5, 0.8, 0.9, 1, 2, 3, 5, 10, 15, 20, 25, 30, 40, 50, 80, 100, 200, 250, 300, 350, 400, 450, 500, 800, 1000],
        'gamma':   ['auto']
        }
    search = GridSearchCV(estimator = model, param_grid = param, verbose=1,
        scoring=('accuracy','precision','recall','f1'), refit='accuracy', n_jobs=24, cv=10)
    search.fit(X[known], y[known])
    return search

def validate(X, y):
    known = y == y
    
    # train
    param = {'C': 2, 'gamma': 'auto', 'kernel': 'rbf'}
    model = SVC(random_state=SEED, **param)
    model.fit(X[known], y[known])
    
    # test
    y_pred = model.predict(X[val_ind])
    val['predict'] = y_pred.astype(int)

    # print('accuracy  :', accuracy_score(y_pred[validation], vali['observed'].values))
    # print('precision :', precision_score(y_pred[validation], vali['observed'].values))
    # print('recall    :', recall_score(y_pred[validation], vali['observed'].values))
    # print('f1        :', f1_score(y_pred[validation], vali['observed'].values))
    # print(y_pred[validation], vali['observed'].values)
    
    print('correct:', sum(val['prenyl'].astype(int) == val['predict']))
    
    all_models['svm_aaindex'] = model.predict(X)
    return val

X      = aai_pc
y      = prenyl
search = parameter_search(X, y)
display(grid_search_summary(search).head(5))
validate(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   7 out of  10 | elapsed:    0.8s remaining:    0.4s
[Parallel(n_jobs=24)]: Done  10 out of  10 | elapsed:    0.9s finished


,accuracy,precision,recall,f1,params,rank_test_accuracy
0,85.1 ± 3.5,86.6 ± 4.1,82.4 ± 3.6,84.4 ± 3.6,"{'C': 2, 'gamma': 'auto', 'kernel': 'rbf'}",1


correct: 14


,sequence,prenyl,cleave,predict
0,CIIS,1,1,1
1,CIKS,0,0,0
2,CIIL,1,1,1
3,CIDL,0,0,0
4,CVIM,1,1,1
5,CVKM,1,0,0
6,CSII,1,1,1
7,CSEI,0,0,0
8,CSNA,1,0,0
9,CYNA,1,0,0


In [10]:
from sklearn.svm import SVC

def parameter_search(X, y):
    known = y == y
    
    # parameter optimization
    model  = SVC(random_state=SEED)
    param  = {
        'kernel' : ['rbf'], #['linear', 'rbf', 'sigmoid'], 
        'C':       [20], #[1, 5, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80, 90, 100, 200, 250],
        'gamma':   ['auto']
        }
    search = GridSearchCV(estimator = model, param_grid = param, verbose=1,
        scoring=('accuracy','precision','recall','f1'), refit='accuracy', n_jobs=23, cv=10)
    search.fit(X[known], y[known])
    return search

def validate(X, y):
    known = y == y
    
    # train
    param = {'C': 20, 'gamma': 'auto', 'kernel': 'rbf'}
    model = SVC(random_state=SEED, **param)
    model.fit(X[known], y[known])
    
    # test
    y_pred = model.predict(X[val_ind])
    val['predict'] = y_pred.astype(int)

    # print('accuracy  :', accuracy_score(y_pred[validation], vali['observed'].values))
    # print('precision :', precision_score(y_pred[validation], vali['observed'].values))
    # print('recall    :', recall_score(y_pred[validation], vali['observed'].values))
    # print('f1        :', f1_score(y_pred[validation], vali['observed'].values))
    # print(y_pred[validation], vali['observed'].values)
    
    print('correct:', sum(val['prenyl'].astype(int) == val['predict']))
    
    all_models['svm_esm1b'] = model.predict(X)
    return val

X      = esm_pc
y      = prenyl
search = parameter_search(X, y)
display(grid_search_summary(search).head(5))
validate(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=23)]: Using backend LokyBackend with 23 concurrent workers.
[Parallel(n_jobs=23)]: Done  10 out of  10 | elapsed:    0.9s finished


,accuracy,precision,recall,f1,params,rank_test_accuracy
0,86.4 ± 3.0,86.6 ± 3.3,85.5 ± 4.1,86.0 ± 3.1,"{'C': 20, 'gamma': 'auto', 'kernel': 'rbf'}",1


correct: 16


,sequence,prenyl,cleave,predict
0,CIIS,1,1,1
1,CIKS,0,0,0
2,CIIL,1,1,1
3,CIDL,0,0,0
4,CVIM,1,1,1
5,CVKM,1,0,0
6,CSII,1,1,1
7,CSEI,0,0,0
8,CSNA,1,0,0
9,CYNA,1,0,0


# train GradientBoost

In [11]:
from sklearn.ensemble import GradientBoostingClassifier 

def parameter_search(X, y):
    known = y == y
    
    # parameter optimization
    model  = GradientBoostingClassifier(random_state=SEED)
    param  = {
        'n_estimators'      : [100],  #range(100,401,20),
        
        'max_depth'         : [6],    #range(2,9,2),
        'min_samples_split' : [200],  #range(100,601,100),
        
        'min_samples_leaf'  : [60], #range(60,101,10),
        
        'max_features'      : [5],   # range(3,27,2),
        'subsample'         : [0.7], # [0.6,0.7,0.75,0.8,0.85,0.9,0.95],
    }
    search = GridSearchCV(estimator = model, param_grid = param, verbose=1,
        scoring=('accuracy','precision','recall','f1'), refit='accuracy', n_jobs=22, cv=10)
    
    search.fit(X[known], y[known])
    return search

def validate(X, y):
    known = y == y
    
    # train
    param = {'max_depth': 6, 'max_features': 5, 'min_samples_leaf': 60, 'min_samples_split': 200, 'n_estimators': 100, 'subsample': 0.7}
    model = GradientBoostingClassifier(random_state=SEED, **param)
    model.fit(X[known], y[known])
    
    # test
    y_pred = model.predict(X[val_ind])
    val['predict'] = y_pred.astype(int)

    # print('accuracy  :', accuracy_score(y_pred[validation], vali['observed'].values))
    # print('precision :', precision_score(y_pred[validation], vali['observed'].values))
    # print('recall    :', recall_score(y_pred[validation], vali['observed'].values))
    # print('f1        :', f1_score(y_pred[validation], vali['observed'].values))
    # print(y_pred[validation], vali['observed'].values)
    
    print('correct:', sum(val['prenyl'].astype(int) == val['predict']))
    
    all_models['gbdt_sequence'] = model.predict(X)
    return val

X      = oh_pc
y      = prenyl
search = parameter_search(X, y)
display(grid_search_summary(search).head(5))
validate(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=22)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=22)]: Done  10 out of  10 | elapsed:    0.8s finished


,accuracy,precision,recall,f1,params,rank_test_accuracy
0,86.2 ± 2.4,87.9 ± 3.5,83.4 ± 3.5,85.5 ± 2.6,"{'max_depth': 6, 'max_features': 5, 'min_samples_leaf': 60, 'min_samples_split': 200, 'n_estimators': 100, 'subsample': 0.7}",1


correct: 13


,sequence,prenyl,cleave,predict
0,CIIS,1,1,1
1,CIKS,0,0,0
2,CIIL,1,1,1
3,CIDL,0,0,0
4,CVIM,1,1,1
5,CVKM,1,0,0
6,CSII,1,1,1
7,CSEI,0,0,0
8,CSNA,1,0,0
9,CYNA,1,0,0


In [12]:
from sklearn.ensemble import GradientBoostingClassifier 

def parameter_search(X, y):
    known = y == y
    
    # parameter optimization
    model  = GradientBoostingClassifier(random_state=SEED)
    param  = {
        'n_estimators'      : [380],  #range(100,401,20),
        
        'max_depth'         : [8],    #range(2,9,2),
        'min_samples_split' : [200],  #range(100,601,100),
        
        'min_samples_leaf'  : [60],   #range(60,101,10),
        'max_features'      : [3],    #range(3,27,2),
        
        'subsample'         : [0.8], #[0.6,0.7,0.75,0.8,0.85,0.9,0.95],
    }
    search = GridSearchCV(estimator = model, param_grid = param, verbose=1,
        scoring=('accuracy','precision','recall','f1'), refit='accuracy', n_jobs=22, cv=10)
    
    search.fit(X[known], y[known])
    return search

def validate(X, y):
    known = y == y
    
    # train
    param = {'max_depth': 8, 'max_features': 3, 'min_samples_leaf': 60, 'min_samples_split': 200, 'n_estimators': 380, 'subsample': 0.8}
    model = GradientBoostingClassifier(random_state=SEED, **param)
    model.fit(X[known], y[known])
    
    # test
    y_pred = model.predict(X[val_ind])
    val['predict'] = y_pred.astype(int)

    # print('accuracy  :', accuracy_score(y_pred[validation], vali['observed'].values))
    # print('precision :', precision_score(y_pred[validation], vali['observed'].values))
    # print('recall    :', recall_score(y_pred[validation], vali['observed'].values))
    # print('f1        :', f1_score(y_pred[validation], vali['observed'].values))
    # print(y_pred[validation], vali['observed'].values)
    
    print('correct:', sum(val['prenyl'].astype(int) == val['predict']))
    
    all_models['gbdt_aaindex'] = model.predict(X)
    return val

X      = aai_pc
y      = prenyl
search = parameter_search(X, y)
display(grid_search_summary(search).head(5))
validate(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=22)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=22)]: Done  10 out of  10 | elapsed:    0.4s finished


,accuracy,precision,recall,f1,params,rank_test_accuracy
0,86.2 ± 2.8,87.2 ± 3.5,84.3 ± 4.3,85.6 ± 3.0,"{'max_depth': 8, 'max_features': 3, 'min_samples_leaf': 60, 'min_samples_split': 200, 'n_estimators': 380, 'subsample': 0.8}",1


correct: 14


,sequence,prenyl,cleave,predict
0,CIIS,1,1,1
1,CIKS,0,0,0
2,CIIL,1,1,1
3,CIDL,0,0,0
4,CVIM,1,1,1
5,CVKM,1,0,0
6,CSII,1,1,1
7,CSEI,0,0,0
8,CSNA,1,0,0
9,CYNA,1,0,0


In [13]:
from sklearn.ensemble import GradientBoostingClassifier 

def parameter_search(X, y):
    known = y == y
    
    # parameter optimization
    model  = GradientBoostingClassifier(random_state=SEED)
    param  = {
        'n_estimators'      : [380],  #range(100,401,20),
        
        'max_depth'         : [2],    #range(2,9,2),
        'min_samples_split' : [500],  #range(100,601,100),
        
        'min_samples_split' : [200],  #range(200,501,50),
        'min_samples_leaf'  : [100],  #range(60,101,10),
        
        'max_features'      : [19],   #range(3,27,2),
        'subsample'         : [0.75], #[0.6,0.7,0.75,0.8,0.85,0.9,0.95],
    }
    search = GridSearchCV(estimator = model, param_grid = param, verbose=1,
        scoring=('accuracy','precision','recall','f1'), refit='accuracy', n_jobs=24, cv=10)
    
    search.fit(X[known], y[known])
    return search

def validate(X, y):
    known = y == y
    
    # train
    param = {'max_depth': 2, 'max_features': 19, 'min_samples_leaf': 100, 'min_samples_split': 200, 'n_estimators': 380, 'subsample': 0.75} 	
    model = GradientBoostingClassifier(random_state=SEED, **param)
    model.fit(X[known], y[known])
    
    # test
    y_pred = model.predict(X[val_ind])
    val['predict'] = y_pred.astype(int)

    # print('accuracy  :', accuracy_score(y_pred[validation], vali['observed'].values))
    # print('precision :', precision_score(y_pred[validation], vali['observed'].values))
    # print('recall    :', recall_score(y_pred[validation], vali['observed'].values))
    # print('f1        :', f1_score(y_pred[validation], vali['observed'].values))
    # print(y_pred[validation], vali['observed'].values)
    
    print('correct:', sum(val['prenyl'].astype(int) == val['predict']))
    
    all_models['gbdt_esm1b'] = model.predict(X)
    return val

X      = esm_pc
y      = prenyl
search = parameter_search(X, y)
display(grid_search_summary(search).head(5))
validate(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   7 out of  10 | elapsed:    1.2s remaining:    0.5s
[Parallel(n_jobs=24)]: Done  10 out of  10 | elapsed:    1.2s finished


,accuracy,precision,recall,f1,params,rank_test_accuracy
0,85.0 ± 2.9,85.8 ± 3.8,83.2 ± 3.6,84.4 ± 3.0,"{'max_depth': 2, 'max_features': 19, 'min_samples_leaf': 100, 'min_samples_split': 200, 'n_estimators': 380, 'subsample': 0.75}",1


correct: 15


,sequence,prenyl,cleave,predict
0,CIIS,1,1,1
1,CIKS,0,0,0
2,CIIL,1,1,1
3,CIDL,0,0,0
4,CVIM,1,1,1
5,CVKM,1,0,1
6,CSII,1,1,1
7,CSEI,0,0,0
8,CSNA,1,0,0
9,CYNA,1,0,0


# train naive bayes

In [14]:
from sklearn.naive_bayes import GaussianNB 

def parameter_search(X, y):
    known = y == y
    
    # parameter optimization
    model  = GaussianNB()
    param  = {
        'var_smoothing': [1e-8], #[1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
    }
    search = GridSearchCV(estimator = model, param_grid = param, verbose=1,
        scoring=('accuracy','precision','recall','f1'), refit='accuracy', n_jobs=22, cv=10)
    
    search.fit(X[known], y[known])
    return search

def validate(X, y):
    known = y == y
    
    # train
    param = {'var_smoothing': 1e-8}
    model = GaussianNB(**param)
    model.fit(X[known], y[known])
    
    # test
    y_pred = model.predict(X[val_ind])
    val['predict'] = y_pred.astype(int)

    # print('accuracy  :', accuracy_score(y_pred[validation], vali['observed'].values))
    # print('precision :', precision_score(y_pred[validation], vali['observed'].values))
    # print('recall    :', recall_score(y_pred[validation], vali['observed'].values))
    # print('f1        :', f1_score(y_pred[validation], vali['observed'].values))
    # print(y_pred[validation], vali['observed'].values)
    
    print('correct:', sum(val['prenyl'].astype(int) == val['predict']))
    
    all_models['naivebayes_sequence'] = model.predict(X)
    return val

X      = oh_pc
y      = prenyl
search = parameter_search(X, y)
display(grid_search_summary(search).head(5))
validate(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=22)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=22)]: Done  10 out of  10 | elapsed:    0.1s finished


,accuracy,precision,recall,f1,params,rank_test_accuracy
0,82.9 ± 1.8,85.5 ± 3.1,78.7 ± 3.3,81.9 ± 1.9,{'var_smoothing': 1e-08},1


correct: 12


,sequence,prenyl,cleave,predict
0,CIIS,1,1,1
1,CIKS,0,0,0
2,CIIL,1,1,1
3,CIDL,0,0,0
4,CVIM,1,1,1
5,CVKM,1,0,0
6,CSII,1,1,1
7,CSEI,0,0,0
8,CSNA,1,0,0
9,CYNA,1,0,0


In [15]:
from sklearn.naive_bayes import GaussianNB 

def parameter_search(X, y):
    known = y == y
    
    # parameter optimization
    model  = GaussianNB()
    param  = {
        'var_smoothing': [1e-8], #[1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
    }
    search = GridSearchCV(estimator = model, param_grid = param, verbose=1,
        scoring=('accuracy','precision','recall','f1'), refit='accuracy', n_jobs=22, cv=10)
    
    search.fit(X[known], y[known])
    return search

def validate(X, y):
    known = y == y
    
    # train
    param = {'var_smoothing': 1e-8}
    model = GaussianNB(**param)
    model.fit(X[known], y[known])
    
    # test
    y_pred = model.predict(X[val_ind])
    val['predict'] = y_pred.astype(int)

    # print('accuracy  :', accuracy_score(y_pred[validation], vali['observed'].values))
    # print('precision :', precision_score(y_pred[validation], vali['observed'].values))
    # print('recall    :', recall_score(y_pred[validation], vali['observed'].values))
    # print('f1        :', f1_score(y_pred[validation], vali['observed'].values))
    # print(y_pred[validation], vali['observed'].values)
    
    print('correct:', sum(val['prenyl'].astype(int) == val['predict']))
    
    all_models['naivebayes_aaindex'] = model.predict(X)
    return val

X      = aai_pc
y      = prenyl
search = parameter_search(X, y)
display(grid_search_summary(search).head(5))
validate(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=22)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=22)]: Done  10 out of  10 | elapsed:    0.3s finished


,accuracy,precision,recall,f1,params,rank_test_accuracy
0,82.1 ± 3.0,82.2 ± 4.0,81.4 ± 3.6,81.7 ± 3.0,{'var_smoothing': 1e-08},1


correct: 14


,sequence,prenyl,cleave,predict
0,CIIS,1,1,1
1,CIKS,0,0,0
2,CIIL,1,1,1
3,CIDL,0,0,0
4,CVIM,1,1,1
5,CVKM,1,0,0
6,CSII,1,1,1
7,CSEI,0,0,0
8,CSNA,1,0,0
9,CYNA,1,0,1


In [16]:
from sklearn.naive_bayes import GaussianNB 

def parameter_search(X, y):
    known = y == y
    
    # parameter optimization
    model  = GaussianNB()
    param  = {
        'var_smoothing': [1e-8], #[1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
    }
    search = GridSearchCV(estimator = model, param_grid = param, verbose=1,
        scoring=('accuracy','precision','recall','f1'), refit='accuracy', n_jobs=22, cv=10)
    
    search.fit(X[known], y[known])
    return search

def validate(X, y):
    known = y == y
    
    # train
    param = {'var_smoothing': 1e-8}
    model = GaussianNB(**param)
    model.fit(X[known], y[known])
    
    # test
    y_pred = model.predict(X[val_ind])
    val['predict'] = y_pred.astype(int)

    # print('accuracy  :', accuracy_score(y_pred[validation], vali['observed'].values))
    # print('precision :', precision_score(y_pred[validation], vali['observed'].values))
    # print('recall    :', recall_score(y_pred[validation], vali['observed'].values))
    # print('f1        :', f1_score(y_pred[validation], vali['observed'].values))
    # print(y_pred[validation], vali['observed'].values)
    
    print('correct:', sum(val['prenyl'].astype(int) == val['predict']))
    
    all_models['naivebayes_esm1b'] = model.predict(X)
    return val

X      = esm_pc
y      = prenyl
search = parameter_search(X, y)
display(grid_search_summary(search).head(5))
validate(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=22)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=22)]: Done  10 out of  10 | elapsed:    0.1s finished


,accuracy,precision,recall,f1,params,rank_test_accuracy
0,73.2 ± 2.3,70.4 ± 1.9,78.3 ± 3.9,74.1 ± 2.5,{'var_smoothing': 1e-08},1


correct: 11


,sequence,prenyl,cleave,predict
0,CIIS,1,1,1
1,CIKS,0,0,1
2,CIIL,1,1,1
3,CIDL,0,0,0
4,CVIM,1,1,1
5,CVKM,1,0,0
6,CSII,1,1,1
7,CSEI,0,0,0
8,CSNA,1,0,1
9,CYNA,1,0,0


# train kNN

In [17]:
from sklearn.neighbors import KNeighborsClassifier 

def parameter_search(X, y):
    known = y == y
    
    # parameter optimization
    model  = KNeighborsClassifier()
    param  = {
        'n_jobs'    : [1],
        'algorithm' : ['kd_tree'],
        'n_neighbors': [14], #[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
        'weights': ['distance'], #['uniform','distance'],
    }
    search = GridSearchCV(estimator = model, param_grid = param, verbose=1,
        scoring=('accuracy','precision','recall','f1'), refit='accuracy', n_jobs=22, cv=10)
    
    search.fit(X[known], y[known])
    return search

def validate(X, y):
    known = y == y
    
    # train
    param = {'algorithm': 'kd_tree', 'n_jobs': 1, 'n_neighbors': 14, 'weights': 'distance'}
    model = KNeighborsClassifier(**param)
    model.fit(X[known], y[known])
    
    # test
    y_pred = model.predict(X[val_ind])
    val['predict'] = y_pred.astype(int)

    # print('accuracy  :', accuracy_score(y_pred[validation], vali['observed'].values))
    # print('precision :', precision_score(y_pred[validation], vali['observed'].values))
    # print('recall    :', recall_score(y_pred[validation], vali['observed'].values))
    # print('f1        :', f1_score(y_pred[validation], vali['observed'].values))
    # print(y_pred[validation], vali['observed'].values)
    
    print('correct:', sum(val['prenyl'].astype(int) == val['predict']))
    
    all_models['knn_sequence'] = model.predict(X)
    return val

X      = oh_pc
y      = prenyl
search = parameter_search(X, y)
display(grid_search_summary(search).head(5))
validate(X, y)

[Parallel(n_jobs=22)]: Using backend LokyBackend with 22 concurrent workers.


Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=22)]: Done  10 out of  10 | elapsed:    0.1s finished


,accuracy,precision,recall,f1,params,rank_test_accuracy
0,84.1 ± 3.7,82.7 ± 4.3,85.5 ± 4.3,84.0 ± 3.7,"{'algorithm': 'kd_tree', 'n_jobs': 1, 'n_neighbors': 14, 'weights': 'distance'}",1


correct: 15


,sequence,prenyl,cleave,predict
0,CIIS,1,1,1
1,CIKS,0,0,0
2,CIIL,1,1,1
3,CIDL,0,0,0
4,CVIM,1,1,1
5,CVKM,1,0,0
6,CSII,1,1,1
7,CSEI,0,0,0
8,CSNA,1,0,0
9,CYNA,1,0,0


In [18]:
from sklearn.neighbors import KNeighborsClassifier 

def parameter_search(X, y):
    known = y == y
    
    # parameter optimization
    model  = KNeighborsClassifier()
    param  = {
        'n_jobs'    : [1],
        'algorithm' : ['kd_tree'],
        'n_neighbors': [22], #[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
        'weights': ['distance'], #['uniform','distance'],
    }
    search = GridSearchCV(estimator = model, param_grid = param, verbose=1,
        scoring=('accuracy','precision','recall','f1'), refit='accuracy', n_jobs=22, cv=10)
    
    search.fit(X[known], y[known])
    return search

def validate(X, y):
    known = y == y
    
    # train
    param = {'algorithm': 'kd_tree', 'n_jobs': 1, 'n_neighbors': 6, 'weights': 'distance'}
    model = KNeighborsClassifier(**param)
    model.fit(X[known], y[known])
    
    # test
    y_pred = model.predict(X[val_ind])
    val['predict'] = y_pred.astype(int)

    # print('accuracy  :', accuracy_score(y_pred[validation], vali['observed'].values))
    # print('precision :', precision_score(y_pred[validation], vali['observed'].values))
    # print('recall    :', recall_score(y_pred[validation], vali['observed'].values))
    # print('f1        :', f1_score(y_pred[validation], vali['observed'].values))
    # print(y_pred[validation], vali['observed'].values)
    
    print('correct:', sum(val['prenyl'].astype(int) == val['predict']))
    
    all_models['knn_aaindex'] = model.predict(X)
    return val

X      = aai_pc
y      = prenyl
search = parameter_search(X, y)
display(grid_search_summary(search).head(5))
validate(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=22)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=22)]: Done  10 out of  10 | elapsed:    0.1s finished


,accuracy,precision,recall,f1,params,rank_test_accuracy
0,82.7 ± 2.3,83.5 ± 3.1,81.0 ± 3.0,82.2 ± 2.3,"{'algorithm': 'kd_tree', 'n_jobs': 1, 'n_neighbors': 22, 'weights': 'distance'}",1


correct: 15


,sequence,prenyl,cleave,predict
0,CIIS,1,1,1
1,CIKS,0,0,0
2,CIIL,1,1,1
3,CIDL,0,0,0
4,CVIM,1,1,1
5,CVKM,1,0,0
6,CSII,1,1,1
7,CSEI,0,0,0
8,CSNA,1,0,0
9,CYNA,1,0,0


In [19]:
from sklearn.neighbors import KNeighborsClassifier 

def parameter_search(X, y):
    known = y == y
    
    # parameter optimization
    model  = KNeighborsClassifier()
    param  = {
        'n_jobs'    : [1],
        'algorithm' : ['kd_tree'],
        'n_neighbors': [6], #[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
        'weights': ['distance'], #['uniform','distance'],
    }
    search = GridSearchCV(estimator = model, param_grid = param, verbose=1,
        scoring=('accuracy','precision','recall','f1'), refit='accuracy', n_jobs=22, cv=10)
    
    search.fit(X[known], y[known])
    return search

def validate(X, y):
    known = y == y
    
    # train
    param = {'algorithm': 'kd_tree', 'n_jobs': 1, 'n_neighbors': 6, 'weights': 'distance'}
    model = KNeighborsClassifier(**param)
    model.fit(X[known], y[known])
    
    # test
    y_pred = model.predict(X[val_ind])
    val['predict'] = y_pred.astype(int)

    # print('accuracy  :', accuracy_score(y_pred[validation], vali['observed'].values))
    # print('precision :', precision_score(y_pred[validation], vali['observed'].values))
    # print('recall    :', recall_score(y_pred[validation], vali['observed'].values))
    # print('f1        :', f1_score(y_pred[validation], vali['observed'].values))
    # print(y_pred[validation], vali['observed'].values)
    
    print('correct:', sum(val['prenyl'].astype(int) == val['predict']))
    
    all_models['knn_esm1b'] = model.predict(X)
    return val

X      = esm_pc
y      = prenyl
search = parameter_search(X, y)
display(grid_search_summary(search).head(5))
validate(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=22)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=22)]: Done  10 out of  10 | elapsed:    0.3s finished


,accuracy,precision,recall,f1,params,rank_test_accuracy
0,83.0 ± 2.3,82.4 ± 3.5,83.4 ± 2.4,82.9 ± 2.1,"{'algorithm': 'kd_tree', 'n_jobs': 1, 'n_neighbors': 6, 'weights': 'distance'}",1


correct: 15


,sequence,prenyl,cleave,predict
0,CIIS,1,1,1
1,CIKS,0,0,0
2,CIIL,1,1,1
3,CIDL,0,0,0
4,CVIM,1,1,1
5,CVKM,1,0,0
6,CSII,1,1,1
7,CSEI,0,0,0
8,CSNA,1,0,0
9,CYNA,1,0,0


# final summary

In [22]:
df = pd.DataFrame(all_models)

In [25]:
df.sort_values('motif').to_csv('results/all_models_prenylation.csv', index=None)